### Image labelling using CNN

This notebook does a transfer learning based on the VGG-16 pre-trained network.

In [1]:
# Import libraries and modules
import numpy as np
import pandas as pd
import pickle

from os import listdir
from os.path import isfile, join

import scipy.io
import PIL.Image
from PIL import ImageTk

from keras.preprocessing.image import load_img as k_load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input

from tkinter import *
import tkinter.filedialog

Using Theano backend.


### Utility methods

#### Reading pictures and storing them into an array

In [2]:
# Method to read a file picture and return it as VGG-16 requires
def load_img_file(picture_file):
    # Load picture and resez all to 224x224 pixels
    img = k_load_img(picture_file, target_size=(224, 224))
    
    # Picture has to be converted into Numpy array
    img = img_to_array(img)
    
    return img

In [3]:
def predict_breed_img(img_file):
    img = load_img_file(img_file)

    # Preprocess input as VGG-16 requires it
    new_picture = preprocess_input(img)
    new_picture = new_picture.reshape(1, 3, 224, 224)

    # Normalize all values to be between 0 and 1
    new_picture = new_picture.astype('float32')

    new_picture /= 255

    # Evaluate model on test data
    new_label = model.predict(new_picture)

    predicted_breed = np.argmax(new_label)
    
    return predicted_breed


### Main execution

In [4]:
# Define data directories
my_data = "../data"
images = my_data + '/Images' # Images folder

In [5]:
# Parameters

# Number of dogs breeds to include
nb_breeds = 10

In [6]:
# Get breed names

folders = [f for f in listdir(images) if (~isfile(join(images, f)) &
                                                f.lower().startswith('n02'))]
breed_id = 0
df_breeds = pd.DataFrame(columns=['breed_id', 'breed_name'])
for folder in folders:
    breed_name = folder[10:]
    df_breeds = df_breeds.append(pd.Series({'breed_id' : breed_id, 'breed_name': breed_name}), ignore_index=True)
    breed_id += 1

In [7]:
df_breeds

,breed_id,breed_name
0,0,Chihuahua
1,1,Japanese_spaniel
2,2,Maltese_dog
3,3,Pekinese
4,4,Shih-Tzu
5,5,Blenheim_spaniel
6,6,papillon
7,7,toy_terrier
8,8,Rhodesian_ridgeback
9,9,Afghan_hound


In [8]:
# restore the model
model_name = 'vgg14-{0}-{1}-{2}.pkl'.format(nb_breeds, 'full', '20')
print('Restoring model {0}'.format(model_name))
input_file = open(model_name, 'rb')
model = pickle.load(input_file)
input_file.close()

Restoring model vgg14-10-full-20.pkl


In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3, 224, 224)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 224, 224)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 224, 224)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 112, 112)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 112, 112)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 112, 112)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 128, 56, 56)       0         
__________

In [10]:
class UI(Frame):
    
    def __init__(self, fenetre, **kwargs):
        Frame.__init__(self, fenetre, width=768, height=576, **kwargs)
        self.pack(fill=BOTH)
        
        self.titre = Label(self, text="Reconnaissance de race de chiens", font=("Arial", 18))
        self.titre.pack()
        
        self.bouton = Button(self, text="Sélectionnez une image", font=("Arial", 12), command=self.predict_breed )
        self.bouton.pack()
        
        # Create area to display the loaded image
        self.canvas = Canvas(self, width=350, height=350)
        self.image_dog = self.canvas.create_image(0, 0, anchor=NW)
        self.canvas.pack()
        
        self.breed = Label(self, text="Réponse ici", font=("Arial", 14))
        self.breed.pack()
        
        self.nb = Label(self, text="Base {0} races".format(nb_breeds), font=("Arial", 8))
        self.nb.pack()

    
    def predict_breed(self):
        self.breed["text"] = "Veuillez patienter..."
        # Load image
        picture_file = tkinter.filedialog.askopenfilename()

        predicted_breed = predict_breed_img(picture_file)
        
        # Load image to display it
        pil_img = PIL.Image.open(picture_file)
        # Resize image to fit into the canvas size
        wpercent = (350/float(pil_img.size[0]))
        hsize = int((float(pil_img.size[1])*float(wpercent)))
        pil_img = pil_img.resize((350,hsize), PIL.Image.ANTIALIAS)
        # Convert image into tkinter compatible format and display it
        myImage = ImageTk.PhotoImage(pil_img)
        self.myImage = myImage  # Avoid Python garbage collector to remove image data
        self.canvas.itemconfig(self.image_dog, image = myImage)
               
        self.breed["text"] = ("Ce chien est probablement de race {0} "
                              .format(df_breeds.loc[predicted_breed, ['breed_name']].values[0]))

In [11]:
# Start graphical User Interface

my_window = Tk()
interface = UI(my_window)
interface.mainloop()
